In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
# import hvplot.pandas  # noqa
import holoviews as hv

hv.extension('bokeh', 'matplotlib')
hv.output(backend='matplotlib', fig='svg')
hv.opts.defaults(
    hv.opts.Curve(show_grid=True, fig_size=150, aspect=1.5, linewidth=1.5),
    # hv.opts.Layout(fig_size=150),
)

In [ ]:
## Data 

# PyDreamer

df = pd.concat([pd.read_csv(f) for f in Path('runs').glob('*.csv')])
df['method'] = 'pydreamer'
# discretize to 1e6 steps
df['env_steps'] = (df['env_steps'] / 1e6).apply(np.ceil) * 1e6  
df = df.groupby(['env', 'method', 'run', 'env_steps'])[['return']].mean().reset_index()
df1 = df

# DreamerV2

df = pd.concat([
    pd.read_csv('../baselines/atari-dreamerv2.csv'),
    pd.read_csv('../baselines/montezuma-dreamerv2.csv')
])
df2 = df

# Union

df = pd.concat([df1, df2])
# aggregate runs
df = df.groupby(['method', 'env', 'env_steps'])['return'].agg(['mean', 'std', 'count']).reset_index()
df['std'] = df['std'].fillna(0)
df = df.rename(columns={'mean': 'return', 'std': 'return_std'})
data = df

In [ ]:
# Pong

env = 'atari_pong'
df_env = data[data['env'] == env]

fig = hv.Overlay([
    hv.Curve(df_method, 'env_steps', 'return', group=env, label=method).opts(
        xlim=(0, 20e6),
        ylim=(-22, 22),
    )
    * 
    hv.Spread(df_method, 'env_steps', ['return', 'return_std']).opts(
        alpha=0.2,
    )
    for method, df_method
    in df_env.groupby('method')
]).opts(title=env)

hv.save(fig, f'figures/{env}.png', dpi=144)
fig

In [ ]:
# Breakout

env = 'atari_breakout'
df_env = data[data['env'] == env]

fig = hv.Overlay([
    hv.Curve(df_method, 'env_steps', 'return', group=env, label=method).opts(
        xlim=(0, 100e6),
        ylim=(0, 500),
    )
    * 
    hv.Spread(df_method, 'env_steps', ['return', 'return_std']).opts(
        alpha=0.2,
    )
    for method, df_method
    in df_env.groupby('method')
]).opts(title=env)

hv.save(fig, f'figures/{env}.png', dpi=144)
fig

In [ ]:
# Montezuma

env = 'atari_montezuma_revenge'
df_env = data[
    (data['env'] == env) &
    data['method'].isin(['pydreamer', 'dreamerv2_g99'])
]

fig = hv.Overlay([
    hv.Curve(df_method, 'env_steps', 'return', group=env, label=method).opts(
        xlim=(0, 200e6),
        ylim=(0, 3000),
    )
    *
    hv.Spread(df_method, 'env_steps', ['return', 'return_std']).opts(
        alpha=0.2,
    )
    for method, df_method
    in df_env.groupby('method')
]).opts(title=env)

hv.save(fig, f'figures/{env}.png', dpi=144)
fig

In [ ]:
# Space invaders

env = 'atari_space_invaders'
df_env = data[
    (data['env'] == env) &
    data['method'].isin(['pydreamer', 'dreamerv2'])
]

fig = hv.Overlay([
    hv.Curve(df_method, 'env_steps', 'return', group=env, label=method).opts(
        xlim=(0, 200e6),
        ylim=(0, 7000),
    )
    *
    hv.Spread(df_method, 'env_steps', ['return', 'return_std']).opts(
        alpha=0.2,
    )
    for method, df_method
    in df_env.groupby('method')
]).opts(title=env)

hv.save(fig, f'figures/{env}.png', dpi=144)
fig